# Data Preprocessing

---

## Purpose

This notebook prepares the raw data for analysis and modeling by cleaning and transforming variables.

- Load and inspect the raw dataset
- Handle missing values through imputation
- Generate summary statistics to understand variable distributions
- Create bivariate reports to check relationships with the target
- Transform key features for better model performance

In [1]:
!pip install -r ../requirements.txt -q

In [2]:
# import packages
import os
import csv
import numpy as np
import pandas as pd
import warnings

# visualization package
import seaborn as sns
import matplotlib.pyplot as plt

# ignore warning
warnings.filterwarnings("ignore")

from utils import *

### Load Data

In [3]:
with open("../data/census-bureau.columns") as f:
    columns = [line.strip() for line in f]

In [4]:
len(columns)

42

In [5]:
df = pd.read_csv(
    "../data/census-bureau.data",
    header=None,
    names=columns,
    sep=",",
    engine="python",
)

In [6]:
# replace " ", ? and NA values with NaN
df = df.replace("", np.nan)
df = df.replace("NA", np.nan)
df = df.replace("none", np.nan)
df = df.replace("?", np.nan)

In [7]:
# number of records and fields in the data
print(df.shape)
print(f"Number of records: {df.shape[0]}")
print(f"Number of fields: {df.shape[1]}")

(199523, 42)
Number of records: 199523
Number of fields: 42


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199523 entries, 0 to 199522
Data columns (total 42 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   age                                         199523 non-null  int64  
 1   class of worker                             199523 non-null  object 
 2   detailed industry recode                    199523 non-null  int64  
 3   detailed occupation recode                  199523 non-null  int64  
 4   education                                   199523 non-null  object 
 5   wage per hour                               199523 non-null  int64  
 6   enroll in edu inst last wk                  199523 non-null  object 
 7   marital stat                                199523 non-null  object 
 8   major industry code                         199523 non-null  object 
 9   major occupation code                       199523 non-null  object 
 

In [9]:
df.label.value_counts()

label
- 50000.    187141
50000+.      12382
Name: count, dtype: int64

In [10]:
df["label"] = df["label"].str.strip()
df["label"] = df["label"].map({"50000+.": 1, "- 50000.": 0})

In [11]:
df.label.value_counts()

label
0    187141
1     12382
Name: count, dtype: int64

In [12]:
# Unweighted event rate
unweighted_rate = df["label"].mean()
print(f"Unweighted event rate: {unweighted_rate:.2%}")

# Weighted event rate
weighted_rate = np.average(df["label"], weights=df["weight"])
print(f"Weighted event rate: {weighted_rate:.2%}")

Unweighted event rate: 6.21%
Weighted event rate: 6.41%


In [13]:
# Clean the string columns
df = clean_string_values(df)

In [14]:
df.columns = df.columns.str.replace(" ", "_").str.lower()

In [15]:
df.shape

(199523, 42)

In [16]:
df.head(2)

,age,class_of_worker,detailed_industry_recode,detailed_occupation_recode,education,wage_per_hour,enroll_in_edu_inst_last_wk,marital_stat,major_industry_code,major_occupation_code,...,country_of_birth_father,country_of_birth_mother,country_of_birth_self,citizenship,own_business_or_self_employed,fill_inc_questionnaire_for_veteran's_admin,veterans_benefits,weeks_worked_in_year,year,label
0,73,not in universe,0,0,high school graduate,0,not in universe,widowed,not in universe or children,not in universe,...,united-states,united-states,united-states,native- born in the united states,0,not in universe,2,0,95,0
1,58,self-employed-not incorporated,4,34,some college but no degree,0,not in universe,divorced,construction,precision production craft & repair,...,united-states,united-states,united-states,native- born in the united states,0,not in universe,2,52,94,0


In [17]:
df.year.value_counts()

year
94    99827
95    99696
Name: count, dtype: int64

### Summarise Data

In [18]:
summary_df = summary_statistics(df)
summary_df["percentage_missing_cnt"] = summary_df["Missing_Count"] * 100 / df.shape[0]
summary_df

,Variable_Name,Variable_Type,Missing_Count,Most_Frequent_Value,Mean,Standard_Deviation,Min,Max,Unique_Values,IQR_Outliers,5th percentile,10th percentile,25th percentile,50th percentile (Median),75th percentile,90th percentile,95th percentile,Percentage_of_Zeros,Percentage_of_Negatives,percentage_missing_cnt
0,age,Continuous,0,34,34.494199,22.310895,0,90,91,0.0,3.0,6.0,15.0,33.0,50.0,67.0,75.0,1.422894,0.0,0.000000
1,class_of_worker,Categorical,0,not in universe,NA,NA,NA,NA,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.000000
2,detailed_industry_recode,Continuous,0,0,15.35232,18.067129,0,51,52,0.0,0.0,0.0,0.0,0.0,33.0,43.0,44.0,50.462353,0.0,0.000000
3,detailed_occupation_recode,Continuous,0,0,11.306556,14.454204,0,46,47,0.0,0.0,0.0,0.0,0.0,26.0,35.0,38.0,50.462353,0.0,0.000000
4,education,Categorical,0,high school graduate,NA,NA,NA,NA,17,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.000000
5,wage_per_hour,Continuous,0,0,55.426908,274.896454,0,9999,1240,5.665512,0.0,0.0,0.0,0.0,0.0,0.0,495.0,94.334488,0.0,0.000000
6,enroll_in_edu_inst_last_wk,Categorical,0,not in universe,NA,NA,NA,NA,3,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.000000
7,marital_stat,Categorical,0,never married,NA,NA,NA,NA,7,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.000000
8,major_industry_code,Categorical,0,not in universe or children,NA,NA,NA,NA,24,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.000000
9,major_occupation_code,Categorical,0,not in universe,NA,NA,NA,NA,15,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.000000


In [19]:
summary_df.to_excel("../artifacts/data_quality/summary_report.xlsx", index=False)

### Mappings and transformations

In [20]:
df["education"].value_counts()

education
high school graduate                      48407
children                                  47422
some college but no degree                27820
bachelors degree(ba ab bs)                19865
7th and 8th grade                          8007
10th grade                                 7557
11th grade                                 6876
masters degree(ma ms meng med msw mba)     6541
9th grade                                  6230
associates degree-occup /vocational        5358
associates degree-academic program         4363
5th or 6th grade                           3277
12th grade no diploma                      2126
1st 2nd 3rd or 4th grade                   1799
prof school degree (md dds dvm llb jd)     1793
doctorate degree(phd edd)                  1263
less than 1st grade                         819
Name: count, dtype: int64

#### 1. Education to Ordinal Variable

In [21]:
education_mapping = {
    "children": 1,
    "less than 1st grade": 2,
    "1st 2nd 3rd or 4th grade": 3,
    "5th or 6th grade": 4,
    "7th and 8th grade": 5,
    "9th grade": 6,
    "10th grade": 7,
    "11th grade": 8,
    "12th grade no diploma": 9,
    "high school graduate": 10,
    "some college but no degree": 11,
    "associates degree-occup /vocational": 12,
    "associates degree-academic program": 13,
    "bachelors degree(ba ab bs)": 14,
    "masters degree(ma ms meng med msw mba)": 15,
    "prof school degree (md dds dvm llb jd)": 16,
    "doctorate degree(phd edd)": 17,
}

df["education_numeric"] = df["education"].map(education_mapping)

if df["education_numeric"].isna().any():
    df["education_numeric"] = df["education_numeric"].fillna(
        df["education_numeric"].median()
    )

display(
    df.groupby(["education", "education_numeric"])
    .size()
    .reset_index(name="count")
    .sort_values(["education_numeric"])
)

df.drop(columns="education", inplace=True)

,education,education_numeric,count
10,children,1,47422
13,less than 1st grade,2,819
3,1st 2nd 3rd or 4th grade,3,1799
4,5th or 6th grade,4,3277
5,7th and 8th grade,5,8007
6,9th grade,6,6230
0,10th grade,7,7557
1,11th grade,8,6876
2,12th grade no diploma,9,2126
12,high school graduate,10,48407


#### 2. Binary flags for `country_of_birth_father`, `country_of_birth_mother`, `country_of_birth_self`

In [22]:
cols_to_transform = [
    "country_of_birth_father",
    "country_of_birth_mother",
    "country_of_birth_self",
]

for col in cols_to_transform:
    new_col_name = f"{col}_is_usa"
    df[new_col_name] = df[col].apply(
        lambda x: 1 if str(x).strip().lower() == "united-states" else 0
    )

    display(df.groupby(f"{col}_is_usa").size().reset_index(name="count"))

    df.drop(columns=col, inplace=True)

,country_of_birth_father_is_usa,count
0,0,40360
1,1,159163


,country_of_birth_mother_is_usa,count
0,0,39044
1,1,160479


,country_of_birth_self_is_usa,count
0,0,22534
1,1,176989


#### 3. Combine categories for `state_of_previous_residence`

In [23]:
df["state_of_previous_residence"] = df["state_of_previous_residence"].apply(
    get_state_mapping
)

display(df.groupby("state_of_previous_residence").size().reset_index(name="count"))

,state_of_previous_residence,count
0,abroad,671
1,not in universe,184458
2,us_state,14394


#### 4. Binary flag for `sex`

In [24]:
df["sex"] = df["sex"].str.strip()
df["sex_is_male"] = df["sex"].map({"male": 1, "female": 0})
df.drop(columns="sex", inplace=True)
df.sex_is_male.value_counts()

sex_is_male
0    103984
1     95539
Name: count, dtype: int64

#### 5.Log Transformation for  `capital_gains`, `capital_losses`, `dividends_from_stocks`

In [25]:
cols_to_transform = ["capital_gains", "capital_losses", "dividends_from_stocks"]

for col in cols_to_transform:
    df[f"has_{col}"] = (df[col] > 0).astype(int)
    display(df.groupby(f"has_{col}").size().reset_index(name="count"))

for col in cols_to_transform:
    df[f"{col}_log"] = np.log1p(df[col])
    df.drop(columns=col, inplace=True)

,has_capital_gains,count
0,0,192144
1,1,7379


,has_capital_losses,count
0,0,195617
1,1,3906


,has_dividends_from_stocks,count
0,0,178382
1,1,21141


### Get Bi-variate analsyis for categorical variables

In [26]:
report_df = bivariate_report(df, "label")

In [27]:
report_df.head(9)

,Variable,Category,Count_1,Count_0,Total,Event_Rate_%,Weighted_Event_Rate_%,Population_%,Weighted_Population_%
0,class_of_worker,federal government,597.0,2328.0,2925.0,20.41,21.08,1.47,1.46
1,class_of_worker,local government,847.0,6937.0,7784.0,10.88,11.22,3.90,3.96
2,class_of_worker,never worked,2.0,437.0,439.0,0.46,0.29,0.22,0.23
3,class_of_worker,not in universe,904.0,99341.0,100245.0,0.90,0.94,50.24,49.21
4,class_of_worker,private,7322.0,64706.0,72028.0,10.17,10.22,36.10,37.20
5,class_of_worker,self-employed-incorporated,1134.0,2131.0,3265.0,34.73,36.26,1.64,1.63
6,class_of_worker,self-employed-not incorporated,1090.0,7355.0,8445.0,12.91,13.29,4.23,4.18
7,class_of_worker,state government,485.0,3742.0,4227.0,11.47,11.72,2.12,2.06
8,class_of_worker,without pay,1.0,164.0,165.0,0.61,0.18,0.08,0.08


In [28]:
report_df.to_excel("../artifacts/data_quality/bivariate_report.xlsx", index=False)

In [29]:
# saving a parquet file
df.to_parquet("../data/processed_data.parquet", engine="pyarrow", index=False)